<h3>This notebook is being used for the capstone assignment of the IBM Data Science Professional Certification.</h3>

Open tasks:<ul><li>streamline</li><li>automate</li><li>clean up</li></ul>

In [1]:
import pandas as pd
import numpy as np

import json

import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

import credentials
import requests
import traceback

from os import mkdir
from os import listdir

from datetime import datetime
from time import sleep

#from selenium import webdriver
#from bs4 import BeautifulSoup as bs

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import preprocessing

from geopy import Point
from geopy.geocoders import ArcGIS
from geopy.distance import geodesic
from geopy.distance import VincentyDistance

from IPython.core.display import display, HTML

In [84]:
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
CLIENT_ID = credentials.credentials['CLIENT_ID'] 
CLIENT_SECRET = credentials.credentials['CLIENT_SECRET']
ACCESS_TOKEN = credentials.credentials['ACCESS_TOKEN']
#VERSION = '20180604'
VERSION = '20200527'

<h3>Import and clean data from census.gov</h3>

I Used this video to help get demographic data from census.gov :https://www.youtube.com/watch?v=K0-ifZS0mQI.

Since the columns were akwardly labeled, and I was unfamiliar with the contents of the file, made sense to do a lot of the cleaning manually in MS Excel.

The CSV file contains socio-economic data from 519 metro/micropolitan areas in the United States.

In [4]:
metro_df = pd.read_csv('DP03_All_Metros.csv')
metro_df.head()

,geographic area name,state,metro-micro,id,population_16_up,unemployment_rate,commute_own_transport_alone,commute_carpool,commute_public,commute_walk,...,household_income_over_200k,median_household_income,mean-median,percent_households_receiving_SS,percent_households_receiving_retirement,mean_retirement_income,percent_private_health_ins,percent_public_health_ins,percent_no_health_ins,percent_in_poverty_last_12_months
0,Aberdeen,WA,Micro,310M400US10140,61308,4.6,79.2,8.9,0.5,3.1,...,2.1,48255,13525,49.6,25.6,22595,53.5,55.7,8.1,9.2
1,Abilene,TX,Metro,310M400US10180,136605,3.7,77.7,13.2,0.1,1.9,...,3.6,48021,18704,31.1,20.7,22444,62.8,33.7,15.9,10.9
2,Adrian,MI,Micro,310M400US10300,79720,3.9,84.6,7.5,0.0,3.1,...,3.3,55378,14948,39.4,25.5,19760,72.2,41.7,4.1,5.9
3,Aguadilla-Isabela,PR,Metro,310M400US10380,249121,14.5,85.4,7.7,0.2,2.3,...,0.7,17064,10214,47.4,14.9,22026,28.9,70.4,5.6,46.9
4,Akron,OH,Metro,310M400US10420,580989,5.5,83.0,7.7,1.6,1.7,...,5.0,60019,19773,33.4,21.5,27158,71.4,36.4,5.5,7.5


<h5>Make sure all Series are correct data type</h5>

In [5]:
metro_df.dtypes

geographic area name                        object
state                                       object
metro-micro                                 object
id                                          object
population_16_up                             int64
unemployment_rate                          float64
commute_own_transport_alone                float64
commute_carpool                            float64
commute_public                             float64
commute_walk                               float64
commute_other                              float64
commute_WFH                                float64
mean_minutes_to_work                       float64
household_income_less_10k                  float64
household_income_10k-15k                   float64
household_income_15k-25k                   float64
household_income_25k-35k                   float64
household_income_35k-50k                   float64
household_income_50k-75k                   float64
household_income_75k-100k      

<h5>The API returns all Null values as -999999999.0. Convert them to the label average.</h5>

In [6]:
metro_df[metro_df['geographic area name'] == 'Fort Payne']

,geographic area name,state,metro-micro,id,population_16_up,unemployment_rate,commute_own_transport_alone,commute_carpool,commute_public,commute_walk,...,household_income_over_200k,median_household_income,mean-median,percent_households_receiving_SS,percent_households_receiving_retirement,mean_retirement_income,percent_private_health_ins,percent_public_health_ins,percent_no_health_ins,percent_in_poverty_last_12_months
160,Fort Payne,AL,Micro,310M400US22840,57239,3.2,-999999999.0,-999999999.0,-999999999.0,-999999999.0,...,3.0,36998,20356,42.2,18.4,15738,60.2,38.7,13.8,18.2


In [7]:
metro_df.replace(-999999999.0, np.nan, inplace = True)

In [8]:
metro_df[metro_df['geographic area name'] == 'Fort Payne']

,geographic area name,state,metro-micro,id,population_16_up,unemployment_rate,commute_own_transport_alone,commute_carpool,commute_public,commute_walk,...,household_income_over_200k,median_household_income,mean-median,percent_households_receiving_SS,percent_households_receiving_retirement,mean_retirement_income,percent_private_health_ins,percent_public_health_ins,percent_no_health_ins,percent_in_poverty_last_12_months
160,Fort Payne,AL,Micro,310M400US22840,57239,3.2,NaN,NaN,NaN,NaN,...,3.0,36998,20356,42.2,18.4,15738,60.2,38.7,13.8,18.2


In [9]:
for column in metro_df.columns[4:]:
    metro_df[column].replace(np.nan, metro_df[column].mean(), inplace = True)

In [10]:
metro_df[metro_df['geographic area name'] == 'Fort Payne']

,geographic area name,state,metro-micro,id,population_16_up,unemployment_rate,commute_own_transport_alone,commute_carpool,commute_public,commute_walk,...,household_income_over_200k,median_household_income,mean-median,percent_households_receiving_SS,percent_households_receiving_retirement,mean_retirement_income,percent_private_health_ins,percent_public_health_ins,percent_no_health_ins,percent_in_poverty_last_12_months
160,Fort Payne,AL,Micro,310M400US22840,57239,3.2,80.386745,9.377583,1.304288,2.523197,...,3.0,36998,20356,42.2,18.4,15738,60.2,38.7,13.8,18.2


In [11]:
for column in metro_df:
    print((metro_df[column] == -999999999.0).value_counts())

False    519
Name: geographic area name, dtype: int64
False    519
Name: state, dtype: int64
False    519
Name: metro-micro, dtype: int64
False    519
Name: id, dtype: int64
False    519
Name: population_16_up, dtype: int64
False    519
Name: unemployment_rate, dtype: int64
False    519
Name: commute_own_transport_alone, dtype: int64
False    519
Name: commute_carpool, dtype: int64
False    519
Name: commute_public, dtype: int64
False    519
Name: commute_walk, dtype: int64
False    519
Name: commute_other, dtype: int64
False    519
Name: commute_WFH, dtype: int64
False    519
Name: mean_minutes_to_work, dtype: int64
False    519
Name: household_income_less_10k, dtype: int64
False    519
Name: household_income_10k-15k, dtype: int64
False    519
Name: household_income_15k-25k, dtype: int64
False    519
Name: household_income_25k-35k, dtype: int64
False    519
Name: household_income_35k-50k, dtype: int64
False    519
Name: household_income_50k-75k, dtype: int64
False    519
Name: househo

<h3>Add coordinates to metro data</h3>

In [12]:
metro_df[['geographic area name', 'state']].head()

,geographic area name,state
0,Aberdeen,WA
1,Abilene,TX
2,Adrian,MI
3,Aguadilla-Isabela,PR
4,Akron,OH


In [13]:
geolocator = ArcGIS(user_agent = 'IBM_DS_Capstone')

In [14]:
def locateMetroCities(cities, states):
    '''
    cities: str, name of metro. Eg: "Seattle-Tacoma-Bellevue", or "Idaho Falls"
    states: str, state codes. Eg: "ID-WA", or "OR"
    '''
    
    city_list = cities.split('-')
    state_list = states.split('-')
    
    lat_long_list = []
    
    for city in city_list:
        
        location = geolocator.geocode(city + ', ' + state_list[0])
        
        #makes sure we have the right city in the right state
        if (location == None) or (location.address.split(', ')[0] != city):
            
            i = 1
            while (i < len(state_list)) and (location == None):
                
                location = geolocator.geocode(city + ', ' + state_list[i])
                i += 1
        
        lat_long_list.append([location.latitude, location.longitude])
    
    return lat_long_list

<h5>Getting all lat-long data often times-out, collect data in batches. Remember loc is inclusive</h5>

In [ ]:
lat_long = metro_df.loc[0:74].apply(lambda x: locateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
lat_long2 = metro_df.loc[75:148].apply(lambda x: locateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
lat_long3 = metro_df.loc[149:222].apply(lambda x: locateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
lat_long4 = metro_df.loc[223:296].apply(lambda x: locateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
lat_long5 = metro_df.loc[297:370].apply(lambda x: locateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
lat_long6 = metro_df.loc[371:444].apply(lambda x: locateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
lat_long7 = metro_df.loc[445:519].apply(lambda x: locateMetroCities(x['geographic area name'], x['state']), axis = 1)

In [ ]:
print(len(lat_long), len(lat_long2), len(lat_long3), len(lat_long4), len(lat_long5), len(lat_long6), len(lat_long7))

<h5>Save lat_long as csv</h5>

In [ ]:
lat_long = pd.concat([lat_long, lat_long2, lat_long3, lat_long4, lat_long5, lat_long6, lat_long7])

In [ ]:
lat_long.to_csv('lat_long.csv', header = False, index = False)

<h5>Load lat_long from csv</h5>

In [15]:
lat_long = pd.read_csv('lat_long.csv', header = None)
lat_long.head()

,0
0,"[[46.97586000000007, -123.81597999999997]]"
1,"[[32.44917000000004, -99.74141999999995]]"
2,"[[41.89731000000006, -84.03638999999998]]"
3,"[[18.459664865000036, -67.12076224699996], [18..."
4,"[[41.08431000000007, -81.51430999999997]]"


In [16]:
metro_df.insert(5, 'lat_long', lat_long)
metro_df.head()

,geographic area name,state,metro-micro,id,population_16_up,lat_long,unemployment_rate,commute_own_transport_alone,commute_carpool,commute_public,...,household_income_over_200k,median_household_income,mean-median,percent_households_receiving_SS,percent_households_receiving_retirement,mean_retirement_income,percent_private_health_ins,percent_public_health_ins,percent_no_health_ins,percent_in_poverty_last_12_months
0,Aberdeen,WA,Micro,310M400US10140,61308,"[[46.97586000000007, -123.81597999999997]]",4.6,79.2,8.9,0.5,...,2.1,48255,13525,49.6,25.6,22595,53.5,55.7,8.1,9.2
1,Abilene,TX,Metro,310M400US10180,136605,"[[32.44917000000004, -99.74141999999995]]",3.7,77.7,13.2,0.1,...,3.6,48021,18704,31.1,20.7,22444,62.8,33.7,15.9,10.9
2,Adrian,MI,Micro,310M400US10300,79720,"[[41.89731000000006, -84.03638999999998]]",3.9,84.6,7.5,0.0,...,3.3,55378,14948,39.4,25.5,19760,72.2,41.7,4.1,5.9
3,Aguadilla-Isabela,PR,Metro,310M400US10380,249121,"[[18.459664865000036, -67.12076224699996], [18...",14.5,85.4,7.7,0.2,...,0.7,17064,10214,47.4,14.9,22026,28.9,70.4,5.6,46.9
4,Akron,OH,Metro,310M400US10420,580989,"[[41.08431000000007, -81.51430999999997]]",5.5,83.0,7.7,1.6,...,5.0,60019,19773,33.4,21.5,27158,71.4,36.4,5.5,7.5


In [17]:
def lat_long_str_to_list_of_floats(lat_long_str):
    
    lat_long_str = lat_long_str.replace('[', '').replace(']', '')    
    lat_long_list = lat_long_str.split(',')
    
    lat_long_pairs_count =  int(len(lat_long_list) / 2)    
    lat_long_pairs = [[] for i in range(lat_long_pairs_count)]
    
    count = 0
    for pair in lat_long_pairs:
        
        pair.append(float(lat_long_list[0 + count]))
        pair.append(float(lat_long_list[1 + count]))
        count += 2
        
    return lat_long_pairs

In [18]:
metro_df['lat_long'] = pd.Series(map(lat_long_str_to_list_of_floats, metro_df['lat_long']))

<h3>Pick a region of interest</h3>

<h5>Pacific Northwest (WA, OR, ID). Needed the plain WA, OR, ID state codes AND the wildcards to get all the relevant metros</h5>

In [19]:
PNW_metro_df = metro_df[(metro_df['state'] == 'WA') | (metro_df['state'] == 'OR') | (metro_df['state'] == 'ID') | (metro_df['state'].str.contains('WA')) == True | (metro_df['state'].str.contains('OR')) | (metro_df['state'].str.contains('ID'))]
PNW_metro_df.reset_index(drop = True, inplace = True)
PNW_metro_df.head()

,geographic area name,state,metro-micro,id,population_16_up,lat_long,unemployment_rate,commute_own_transport_alone,commute_carpool,commute_public,...,household_income_over_200k,median_household_income,mean-median,percent_households_receiving_SS,percent_households_receiving_retirement,mean_retirement_income,percent_private_health_ins,percent_public_health_ins,percent_no_health_ins,percent_in_poverty_last_12_months
0,Aberdeen,WA,Micro,310M400US10140,61308,"[[46.97586000000007, -123.81597999999997]]",4.6,79.2,8.9,0.5,...,2.1,48255,13525,49.6,25.6,22595,53.5,55.7,8.1,9.2
1,Albany,OR,Metro,310M400US10540,102520,"[[44.63548000000003, -123.10775999999998]]",6.0,79.3,11.9,0.1,...,3.3,54113,14876,37.6,21.0,22918,62.0,45.1,7.0,9.5
2,Bellingham,WA,Metro,310M400US13380,186211,"[[48.752350000000035, -122.47121999999996]]",4.7,76.9,9.4,2.2,...,4.6,62268,15570,33.5,19.9,26216,69.8,38.0,5.8,8.7
3,Bend-Redmond,OR,Metro,310M400US13460,156933,"[[44.05782000000005, -121.31536999999997], [44...",4.7,75.2,8.3,0.1,...,8.0,67577,24693,35.9,22.0,31550,67.4,37.8,8.7,7.1
4,Boise City,ID,Metro,310M400US14260,570659,"[[43.60764000000006, -116.19339999999994]]",3.8,81.1,8.2,0.3,...,5.2,61407,17433,28.7,19.4,23742,72.0,30.1,10.5,6.6


<h3>Get restaurant type frequescies for each metro</h3>

<h5>Get category JSON from Foursquare</h5>

In [20]:
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET,
                VERSION)

categories = requests.get(url).json()

food_category_dict = categories['response']['categories'][3]

<h5>Get all 'id' values from returned JSON</h5>

In [21]:
def gen_dict_extract(key, var):
    '''
    https://stackoverflow.com/questions/9807634/find-all-occurrences-of-a-key-in-nested-dictionaries-and-lists
    Title: Find all occurrences of a key in nested dictionaries and lists
    by user: hexerei software
    28 MAY 2020
    
    Pulls all values with a specific key from a nested dictionary
    'var' is the dict
    
    '''
    if hasattr(var,'items'):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, dict):
                for result in gen_dict_extract(key, v):
                    yield result
            elif isinstance(v, list):
                for d in v:
                    for result in gen_dict_extract(key, d):
                        yield result

In [22]:
id_list = list(gen_dict_extract('id', food_category_dict))
name_list = list(gen_dict_extract('name', food_category_dict))

#remove generic 'Food' category
id_list = id_list[1:]
name_list = name_list[1:]

In [23]:
name_id_dict = {}
id_name_dict = {}
for i in range(0,len(name_list)):
    name_id_dict[name_list[i]] = id_list[i]
    id_name_dict[id_list[i]] = name_list[i]

In [24]:
def getRestaurantCoordsAndType(lat_long_list, category_list, limit = 50, radius = 25000, redo50 = False):

    allVenues_df = pd.DataFrame(columns = ['name', 'category', 'category_id', 'id', 'lat', 'long'])
    
    for lat_long in lat_long_list:
        
        for category in category_list:

            lat = lat_long[0]
            long = lat_long[1]

            url = 'https://api.foursquare.com/v2/venues/explore?&openNow=0&time=any&sortByPopularity=1&categoryId=' + category + '&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                        CLIENT_ID, 
                        CLIENT_SECRET, 
                        VERSION, 
                        lat, 
                        long, 
                        radius, 
                        limit)
            
            result = requests.get(url).json()
            
            #Keep relevant data only
            try:
                #Foursquare will only return a max of 50 venues.
                #If the max is reached, this section splits the initial radial search into several smaller searches
                if 'groups' in result['response'] and len(result['response']['groups'][0]['items']) == 50 and redo50 == False:
                    
                    lat_long_expanded_list = []
                    degree_list = [0, 45, 90, 135, 180, 225, 270, 315]                    
                    for degree in degree_list:
                        if degree % 90 == 0:
                            distance = radius/2
                        else:
                            distance = radius * (2/5)
                        origin = Point(lat, long)
                        lat_long_expanded_list.append(list(geodesic(meters = distance).destination(origin, degree))[0:2])
                    
                    allVenues_df = pd.concat([allVenues_df, getRestaurantCoordsAndType(lat_long_expanded_list, [category], limit = 50, radius = radius/2, redo50 = True)], sort = False)
                    
                elif 'groups' in result['response'] and len(result['response']['groups'][0]['items']) > 0:
                    
                    venues_df = pd.DataFrame(list(gen_dict_extract('venue', result['response']['groups'][0])))
                    
                    venues_df.drop(venues_df.loc[venues_df['categories'].map(lambda x: len(x)== 0)].index, inplace = True)                    
                    venues_df['category'] = venues_df['categories'].map(lambda x: x[0]['name'])
                    venues_df['category_id'] = venues_df['categories'].map(lambda x:x[0]['id'])
                    
                    venues_df['lat'] = venues_df['location'].map(lambda x: x['lat'])
                    venues_df['long'] = venues_df['location'].map(lambda x: x['lng'])
                    
                    venues_df.drop(['categories','location', 'photos'], axis = 1, inplace = True)

                    if 'venuePage' in venues_df:
                        venues_df.drop(['venuePage'], axis = 1, inplace = True)
                    if 'delivery' in venues_df:
                        venues_df.drop(['delivery'], axis = 1, inplace = True)
                    if 'events' in venues_df:
                        venues_df.drop(['events'], axis = 1, inplace = True)
                    
                    allVenues_df = pd.concat([allVenues_df, venues_df], sort = False)
                    
            except KeyError as err:

                print('KeyError for cat: ' + category + ' at ' + str(lat) + ', ' + str(long))
                traceback.print_exc()
                
                continue
                
            except IndexError as err:
                
                print('IndexError for cat: ' + category + ' at ' + str(lat) + ', ' + str(long))
                traceback.print_exc()
            
            except TypeError as err:
                
                print('TypeError for cat: ' + category + ' at ' + str(lat) + ', ' + str(long))
                traceback.print_exc()
    
    allVenues_df.drop_duplicates(subset = 'id', keep = 'last', inplace = True)
    
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Gas Station']
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Grocery Store']
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Food Court']
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Food']
    #allVenues_df.drop(allVenues_df.loc[allVenues_df['category'] == 'Fast Food Restaurant'].index, inplace = True)
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Fast Food Restaurant']
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Coffee Shop']
    allVenues_df = allVenues_df[allVenues_df['category'] != 'Shopping Plaza']
    
    if redo50 == False:
        allVenues_df.rename(columns = {'id' : 'venue_id'}, inplace = True)
    allVenues_df.reset_index(drop = True, inplace = True)
    
    return allVenues_df

<h5>Foursquare only allows 5000 API calls an hour, so batch the calls and delay between batches</h5>

In [568]:
metro_venue_dict = {}
metro_venue_dict2 = {}
metro_venue_dict3 = {}
metro_venue_dict4 = {}
metro_venue_dict5 = {}

print('Starting batch 1 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[0 : 4].index:    
    metro_venue_dict[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

print('Batch complete. 45min cooldown started at ' + datetime.now().strftime("%H:%M:%S") + '\n\n')
sleep(60*45)

print('Starting batch 2 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[5 : 9].index:    
    metro_venue_dict2[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

print('Batch complete. 45min cooldown started at ' + datetime.now().strftime("%H:%M:%S") + '\n\n')
sleep(60*45)

print('Starting batch 3 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[10 : 14].index:    
    metro_venue_dict3[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

print('Batch complete. 45min cooldown started at ' + datetime.now().strftime("%H:%M:%S") + '\n\n')
sleep(60*45)

print('Starting batch 4 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[15 : 19].index:    
    metro_venue_dict4[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

print('Batch complete. 45min cooldown started at ' + datetime.now().strftime("%H:%M:%S") + '\n\n')
sleep(60*45)

print('Starting batch 5 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[20 : 24].index:    
    metro_venue_dict5[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

print('Batch complete. 45min cooldown started at ' + datetime.now().strftime("%H:%M:%S") + '\n\n')
sleep(60*45)

print('Starting batch 6 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[25 : 29].index:    
    metro_venue_dict6[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

print('Batch complete. 45min cooldown started at ' + datetime.now().strftime("%H:%M:%S") + '\n\n')
sleep(60*45)

print('Starting batch 7 at ' + datetime.now().strftime("%H:%M:%S") + '\n')
for i in PNW_metro_df.loc[30 : 34].index:    
    metro_venue_dict7[PNW_metro_df['geographic area name'][i]] = getRestaurantCoordsAndType(PNW_metro_df['lat_long'][i], id_list)

    
print('Final batch complete at ' + datetime.now().strftime("%H:%M:%S"))

Starting batch 1 at 11:15:29

Batch complete. 45min cooldown started at 11:53:55


Starting batch 2 at 12:38:55

Batch complete. 45min cooldown started at 13:00:58


Starting batch 3 at 13:45:58

Batch complete. 45min cooldown started at 14:14:20


Starting batch 4 at 14:59:20

Batch complete. 45min cooldown started at 15:44:43


Starting batch 5 at 16:29:43

Final batch complete


<h5>Combine all dicts</h5>

In [25]:
metro_venue_dict = {**metro_venue_dict, **metro_venue_dict2, **metro_venue_dict3, **metro_venue_dict4, **metro_venue_dict5, **metro_venue_dict6, **metro_venue_dict7}

NameError: name 'metro_venue_dict' is not defined

In [607]:
#mkdir('./metro_venues')
    
for key in metro_venue_dict.keys():
    metro_venue_dict[key].to_csv('./metro_venues/' + key + '.csv', index = False)

<h5>Recreate dict from csv files</h5>

In [25]:
metro_venue_dict = {}

for file in listdir('./metro_venues'):
    metro_venue_dict[file[0:-4]] = pd.read_csv('./metro_venues/' + file)

metro_venue_dict['Aberdeen'].head()

,name,category,category_id,venue_id,lat,long
0,Farm to Fork,American Restaurant,4bf58dd8d48988d14e941735,5691bcd6498e98773ab2eefe,46.915545,-123.971025
1,Light House Drive In,American Restaurant,4bf58dd8d48988d14e941735,538cc275498e4ac4370195d0,46.974977,-123.842533
2,Hong Kong Restaurant,Chinese Restaurant,4bf58dd8d48988d145941735,4c81a0fcd34ca1437ea92480,46.955405,-123.770918
3,Happy Teriyaki,Chinese Restaurant,4bf58dd8d48988d145941735,4c571d2ab7a31b8db6834fda,46.976606,-123.804716
4,Chinese Village,Chinese Restaurant,4bf58dd8d48988d145941735,4c489a4e417b20a18f89e1a9,46.974862,-123.846497


<h3>Cluster Metros</h3>

<h5>Calculate frequency of restaurant types in each metro</h5>

In [26]:
venue_frequency_df = pd.DataFrame(name_list)
venue_frequency_df.set_index(0, inplace = True)
del venue_frequency_df.index.name

for key in metro_venue_dict.keys():
    
    venue_frequency_df[key] = metro_venue_dict[key]['category'].value_counts()

venue_frequency_df.replace(np.nan, 0, inplace = True)

venue_frequency_df.head()

,Aberdeen,Albany,Bellingham,Bend-Redmond,Boise City,Bremerton-Silverdale,Centralia,Coeur d'Alene,Coos Bay,Corvallis,...,Portland-Vancouver-Hillsboro,Roseburg,Salem,Seattle-Tacoma-Bellevue,Shelton,Spokane-Spokane Valley,Twin Falls,Walla Walla,Wenatchee,Yakima
Afghan Restaurant,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
African Restaurant,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0
Ethiopian Restaurant,0.0,0.0,1.0,0.0,3.0,19.0,0.0,0.0,0.0,0.0,...,14.0,0.0,0.0,41.0,0.0,1.0,0.0,1.0,0.0,0.0
American Restaurant,7.0,32.0,31.0,30.0,50.0,70.0,19.0,39.0,17.0,25.0,...,162.0,16.0,50.0,196.0,19.0,84.0,17.0,9.0,10.0,23.0
New American Restaurant,1.0,3.0,5.0,14.0,11.0,46.0,0.0,3.0,1.0,4.0,...,46.0,2.0,1.0,83.0,2.0,7.0,0.0,1.0,3.0,4.0


<h5>Normalise frequencies</h5>

In [27]:
min_max_scaler = preprocessing.MinMaxScaler()
frequency_normed_df = pd.DataFrame(min_max_scaler.fit_transform(venue_frequency_df))

frequency_normed_df.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.000000,0.00000,0.000000,0.000000,0.022727,0.014493,0.00,0.000000,0.000000,0.000000,...,0.003300,0.000000,0.013158,0.017442,0.00,0.00,0.00000,0.000000,0.00,0.000000
1,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.025641,0.000000,0.000000,...,0.003300,0.000000,0.000000,0.011628,0.00,0.02,0.00000,0.000000,0.00,0.000000
2,0.000000,0.00000,0.022727,0.000000,0.034091,0.137681,0.00,0.000000,0.000000,0.000000,...,0.046205,0.000000,0.000000,0.119186,0.00,0.01,0.00000,0.055556,0.00,0.000000
3,0.500000,1.00000,0.704545,0.714286,0.568182,0.507246,0.95,1.000000,1.000000,0.862069,...,0.534653,0.842105,0.657895,0.569767,0.76,0.84,0.62963,0.500000,0.50,0.511111
4,0.071429,0.09375,0.113636,0.333333,0.125000,0.333333,0.00,0.076923,0.058824,0.137931,...,0.151815,0.105263,0.013158,0.241279,0.08,0.07,0.00000,0.055556,0.15,0.088889


<h5>Fit K-means model</h5>

In [28]:
k = 3

kmeans = KMeans(n_clusters = k).fit(frequency_normed_df.transpose())
kmeans.labels_[0:5]#sneakpeek

array([0, 1, 1, 1, 1])

<h5>Insert cluster labels into metro_df</h5>

In [29]:
try:
    PNW_metro_df.insert(0, 'cluster', kmeans.labels_)
except ValueError:
    PNW_metro_df['cluster'] = kmeans.labels_
    
PNW_metro_df.head()

,cluster,geographic area name,state,metro-micro,id,population_16_up,lat_long,unemployment_rate,commute_own_transport_alone,commute_carpool,...,household_income_over_200k,median_household_income,mean-median,percent_households_receiving_SS,percent_households_receiving_retirement,mean_retirement_income,percent_private_health_ins,percent_public_health_ins,percent_no_health_ins,percent_in_poverty_last_12_months
0,0,Aberdeen,WA,Micro,310M400US10140,61308,"[[46.97586000000007, -123.81597999999997]]",4.6,79.2,8.9,...,2.1,48255,13525,49.6,25.6,22595,53.5,55.7,8.1,9.2
1,1,Albany,OR,Metro,310M400US10540,102520,"[[44.63548000000003, -123.10775999999998]]",6.0,79.3,11.9,...,3.3,54113,14876,37.6,21.0,22918,62.0,45.1,7.0,9.5
2,1,Bellingham,WA,Metro,310M400US13380,186211,"[[48.752350000000035, -122.47121999999996]]",4.7,76.9,9.4,...,4.6,62268,15570,33.5,19.9,26216,69.8,38.0,5.8,8.7
3,1,Bend-Redmond,OR,Metro,310M400US13460,156933,"[[44.05782000000005, -121.31536999999997], [44...",4.7,75.2,8.3,...,8.0,67577,24693,35.9,22.0,31550,67.4,37.8,8.7,7.1
4,1,Boise City,ID,Metro,310M400US14260,570659,"[[43.60764000000006, -116.19339999999994]]",3.8,81.1,8.2,...,5.2,61407,17433,28.7,19.4,23742,72.0,30.1,10.5,6.6


In [30]:
PNW_metro_df['cluster'].value_counts()

0    21
1    11
2     3
Name: cluster, dtype: int64

<h3>Visualise the metro clusters</h3>

<h5>Colour-code and overlay the metros on a map</h5>

In [31]:
map_clusters = folium.Map(width = '100%', height = '100%', location = [45.3, -118], zoom_start = 6, tiles = "Stamen Toner")

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for i in PNW_metro_df.index:
    lat = PNW_metro_df['lat_long'][i][0][0]
    long = PNW_metro_df['lat_long'][i][0][1]
    metro = PNW_metro_df['geographic area name'][i]
    pop = PNW_metro_df['population_16_up'][i]
    cluster = PNW_metro_df['cluster'][i]
    
    label = folium.Popup(str(metro) + '\nPopulation: ' + str(pop) + '\nCluster: ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, long],
        radius=3,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The map above shows the various metro/micropolitan areas in Washington, Oregon, and Idaho. The areas have been K-means clustered according to the ratio of different restaurant types in each of them.

<h3>Calculate mean and std of each cluster<h3>

In [32]:
frequency_normed_df.columns = venue_frequency_df.columns

In [33]:
frequency_normed_df.head()

,Aberdeen,Albany,Bellingham,Bend-Redmond,Boise City,Bremerton-Silverdale,Centralia,Coeur d'Alene,Coos Bay,Corvallis,...,Portland-Vancouver-Hillsboro,Roseburg,Salem,Seattle-Tacoma-Bellevue,Shelton,Spokane-Spokane Valley,Twin Falls,Walla Walla,Wenatchee,Yakima
0,0.000000,0.00000,0.000000,0.000000,0.022727,0.014493,0.00,0.000000,0.000000,0.000000,...,0.003300,0.000000,0.013158,0.017442,0.00,0.00,0.00000,0.000000,0.00,0.000000
1,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.025641,0.000000,0.000000,...,0.003300,0.000000,0.000000,0.011628,0.00,0.02,0.00000,0.000000,0.00,0.000000
2,0.000000,0.00000,0.022727,0.000000,0.034091,0.137681,0.00,0.000000,0.000000,0.000000,...,0.046205,0.000000,0.000000,0.119186,0.00,0.01,0.00000,0.055556,0.00,0.000000
3,0.500000,1.00000,0.704545,0.714286,0.568182,0.507246,0.95,1.000000,1.000000,0.862069,...,0.534653,0.842105,0.657895,0.569767,0.76,0.84,0.62963,0.500000,0.50,0.511111
4,0.071429,0.09375,0.113636,0.333333,0.125000,0.333333,0.00,0.076923,0.058824,0.137931,...,0.151815,0.105263,0.013158,0.241279,0.08,0.07,0.00000,0.055556,0.15,0.088889


<h5>Calculate mean and standard dev of each cluster. Create DF for each cluster<h5>

In [34]:
cluster_mean_std = [[[],[]], [[],[]], [[],[]]]

cluster_mean_std[0][0] = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 0]['geographic area name']].mean(axis = 1)
cluster_mean_std[0][1] = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 0]['geographic area name']].std(axis = 1)

cluster_mean_std[1][0] = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 1]['geographic area name']].mean(axis = 1)
cluster_mean_std[1][1] = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 1]['geographic area name']].std(axis = 1)

cluster_mean_std[2][0] = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 2]['geographic area name']].mean(axis = 1)
cluster_mean_std[2][1] = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 2]['geographic area name']].std(axis = 1)

In [35]:
cluster0_mean = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 0]['geographic area name']].mean(axis = 1)
cluster1_mean = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 1]['geographic area name']].mean(axis = 1)
cluster2_mean = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 2]['geographic area name']].mean(axis = 1)

cluster0_std = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 0]['geographic area name']].std(axis = 1)
cluster1_std = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 1]['geographic area name']].std(axis = 1)
cluster2_std = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 2]['geographic area name']].std(axis = 1)

In [36]:
cluster0_df = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 0]['geographic area name']]
cluster1_df = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 1]['geographic area name']]
cluster2_df = frequency_normed_df[PNW_metro_df[PNW_metro_df['cluster'] == 2]['geographic area name']]

cluster0_df.insert(0,'cluster_std', cluster0_std)
cluster0_df.insert(0,'cluster_mean', cluster0_mean)
cluster1_df.insert(0,'cluster_std', cluster1_std)
cluster1_df.insert(0,'cluster_mean', cluster1_mean)
cluster2_df.insert(0,'cluster_std', cluster2_std)
cluster2_df.insert(0,'cluster_mean', cluster2_mean)

In [37]:
cluster0_df.head()

,cluster_mean,cluster_std,Aberdeen,Centralia,Coeur d'Alene,Coos Bay,Grants Pass,Hermiston-Pendleton,Idaho Falls,Kennewick-Richland,...,Medford,Moses Lake,Mount Vernon-Anacortes,Oak Harbor,Pocatello,Roseburg,Twin Falls,Walla Walla,Wenatchee,Yakima
0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000
1,0.004396,0.015323,0.000000,0.00,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.00000,0.000000,0.00,0.000000
2,0.002646,0.012123,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.055556,0.00,0.000000
3,0.765978,0.228284,0.500000,0.95,1.000000,1.000000,0.833333,0.733333,0.814815,0.550000,...,0.818182,0.266667,1.000000,1.000000,1.000000,0.842105,0.62963,0.500000,0.50,0.511111
4,0.052939,0.054898,0.071429,0.00,0.076923,0.058824,0.000000,0.066667,0.037037,0.016667,...,0.072727,0.000000,0.020833,0.090909,0.200000,0.105263,0.00000,0.055556,0.15,0.088889


<h5>Index cluster DFs with restaurant types</h5>

In [38]:
cluster0_df.index = venue_frequency_df.index
cluster1_df.index = venue_frequency_df.index
cluster2_df.index = venue_frequency_df.index

cluster0_df.head()

,cluster_mean,cluster_std,Aberdeen,Centralia,Coeur d'Alene,Coos Bay,Grants Pass,Hermiston-Pendleton,Idaho Falls,Kennewick-Richland,...,Medford,Moses Lake,Mount Vernon-Anacortes,Oak Harbor,Pocatello,Roseburg,Twin Falls,Walla Walla,Wenatchee,Yakima
Afghan Restaurant,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000
African Restaurant,0.004396,0.015323,0.000000,0.00,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.00000,0.000000,0.00,0.000000
Ethiopian Restaurant,0.002646,0.012123,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.055556,0.00,0.000000
American Restaurant,0.765978,0.228284,0.500000,0.95,1.000000,1.000000,0.833333,0.733333,0.814815,0.550000,...,0.818182,0.266667,1.000000,1.000000,1.000000,0.842105,0.62963,0.500000,0.50,0.511111
New American Restaurant,0.052939,0.054898,0.071429,0.00,0.076923,0.058824,0.000000,0.066667,0.037037,0.016667,...,0.072727,0.000000,0.020833,0.090909,0.200000,0.105263,0.00000,0.055556,0.15,0.088889


<h3>Observe what restaurant types are most under represented in what cities</h3>
    <h5>Calculate how many stds each restaurant type frequency is away from the mean in each city</h5>

In [39]:
std_list = [[],[],[]]

In [40]:
#subtract mean from frequecy
std0_df = cluster0_df[cluster0_df.columns[2:23]].subtract(cluster0_df['cluster_mean'], axis = 0)
std1_df = cluster1_df[cluster1_df.columns[2:23]].subtract(cluster1_df['cluster_mean'], axis = 0)
std2_df = cluster2_df[cluster2_df.columns[2:23]].subtract(cluster2_df['cluster_mean'], axis = 0)

#divide by std
std0_df = std0_df[std0_df.columns[:]].div(cluster0_df['cluster_std'], axis = 0)
std1_df = std1_df[std1_df.columns[:]].div(cluster1_df['cluster_std'], axis = 0)
std2_df = std2_df[std2_df.columns[:]].div(cluster2_df['cluster_std'], axis = 0)

std_list[0] = std0_df
std_list[1] = std1_df
std_list[2] = std2_df

#zero out NaNs
# std0_df.replace(np.nan, 0, inplace = True)
# std1_df.replace(np.nan, 0, inplace = True)
# std2_df.replace(np.nan, 0, inplace = True)

In [41]:
std0_df.head()

,Aberdeen,Centralia,Coeur d'Alene,Coos Bay,Grants Pass,Hermiston-Pendleton,Idaho Falls,Kennewick-Richland,Klamath Falls,Lewiston,...,Medford,Moses Lake,Mount Vernon-Anacortes,Oak Harbor,Pocatello,Roseburg,Twin Falls,Walla Walla,Wenatchee,Yakima
Afghan Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
African Restaurant,-0.286855,-0.286855,1.386465,-0.286855,-0.286855,-0.286855,-0.286855,-0.286855,-0.286855,-0.286855,...,-0.286855,-0.286855,-0.286855,-0.286855,4.063777,-0.286855,-0.286855,-0.286855,-0.286855,-0.286855
Ethiopian Restaurant,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,...,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,4.364358,-0.218218,-0.218218
American Restaurant,-1.165117,0.806108,1.025133,1.025133,0.295050,-0.143000,0.213929,-0.946092,1.025133,1.025133,...,0.228679,-2.187234,1.025133,1.025133,1.025133,0.333475,-0.597274,-1.165117,-1.165117,-1.116445
New American Restaurant,0.336796,-0.964322,0.436882,0.107187,-0.964322,0.250055,-0.289668,-0.660728,-0.964322,-0.964322,...,0.360453,-0.964322,-0.584829,0.691647,2.678810,0.953116,-0.964322,0.047659,1.768027,0.654848


In [42]:
std_list[0].head()

,Aberdeen,Centralia,Coeur d'Alene,Coos Bay,Grants Pass,Hermiston-Pendleton,Idaho Falls,Kennewick-Richland,Klamath Falls,Lewiston,...,Medford,Moses Lake,Mount Vernon-Anacortes,Oak Harbor,Pocatello,Roseburg,Twin Falls,Walla Walla,Wenatchee,Yakima
Afghan Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
African Restaurant,-0.286855,-0.286855,1.386465,-0.286855,-0.286855,-0.286855,-0.286855,-0.286855,-0.286855,-0.286855,...,-0.286855,-0.286855,-0.286855,-0.286855,4.063777,-0.286855,-0.286855,-0.286855,-0.286855,-0.286855
Ethiopian Restaurant,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,...,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,-0.218218,4.364358,-0.218218,-0.218218
American Restaurant,-1.165117,0.806108,1.025133,1.025133,0.295050,-0.143000,0.213929,-0.946092,1.025133,1.025133,...,0.228679,-2.187234,1.025133,1.025133,1.025133,0.333475,-0.597274,-1.165117,-1.165117,-1.116445
New American Restaurant,0.336796,-0.964322,0.436882,0.107187,-0.964322,0.250055,-0.289668,-0.660728,-0.964322,-0.964322,...,0.360453,-0.964322,-0.584829,0.691647,2.678810,0.953116,-0.964322,0.047659,1.768027,0.654848


<h3>Make reccomendations on what cities would be best for what restaurants</h3>
<h5>Calculate population/restaurant</h5>

In [43]:
pop_venue_ratio_list = []

for metro in PNW_metro_df['geographic area name']:
    
    pop_venue_ratio_list.append(float(PNW_metro_df[PNW_metro_df['geographic area name'] == metro]['population_16_up']) / len(metro_venue_dict[metro]))

In [44]:
PNW_metro_df.insert(5, 'pop_venue_ratio', pop_venue_ratio_list)

PNW_metro_df.head()

,cluster,geographic area name,state,metro-micro,id,pop_venue_ratio,population_16_up,lat_long,unemployment_rate,commute_own_transport_alone,...,household_income_over_200k,median_household_income,mean-median,percent_households_receiving_SS,percent_households_receiving_retirement,mean_retirement_income,percent_private_health_ins,percent_public_health_ins,percent_no_health_ins,percent_in_poverty_last_12_months
0,0,Aberdeen,WA,Micro,310M400US10140,595.223301,61308,"[[46.97586000000007, -123.81597999999997]]",4.6,79.2,...,2.1,48255,13525,49.6,25.6,22595,53.5,55.7,8.1,9.2
1,1,Albany,OR,Metro,310M400US10540,248.232446,102520,"[[44.63548000000003, -123.10775999999998]]",6.0,79.3,...,3.3,54113,14876,37.6,21.0,22918,62.0,45.1,7.0,9.5
2,1,Bellingham,WA,Metro,310M400US13380,344.835185,186211,"[[48.752350000000035, -122.47121999999996]]",4.7,76.9,...,4.6,62268,15570,33.5,19.9,26216,69.8,38.0,5.8,8.7
3,1,Bend-Redmond,OR,Metro,310M400US13460,291.697026,156933,"[[44.05782000000005, -121.31536999999997], [44...",4.7,75.2,...,8.0,67577,24693,35.9,22.0,31550,67.4,37.8,8.7,7.1
4,1,Boise City,ID,Metro,310M400US14260,537.850141,570659,"[[43.60764000000006, -116.19339999999994]]",3.8,81.1,...,5.2,61407,17433,28.7,19.4,23742,72.0,30.1,10.5,6.6


<h5>Find top 3 metros with highest population/venue ratios</h5>

In [45]:
top_pop_venue_ratio_df = PNW_metro_df.nlargest(3, ["pop_venue_ratio"], keep = 'all')[['cluster', 'geographic area name', 'pop_venue_ratio']]
top_pop_venue_ratio_df.reset_index(inplace = True, drop = True)
top_pop_venue_ratio_df

,cluster,geographic area name,pop_venue_ratio
0,0,Moses Lake,748.114583
1,0,Aberdeen,595.223301
2,1,Boise City,537.850141


<h5>Find 3 least represented restaurant types in each above city</h5>

In [46]:
print(std_list[top_pop_venue_ratio_df.iloc[0]['cluster']].nsmallest(3, [top_pop_venue_ratio_df.iloc[0]['geographic area name']], keep = 'all')[top_pop_venue_ratio_df.iloc[0]['geographic area name']], '\n')
print(std_list[top_pop_venue_ratio_df.iloc[1]['cluster']].nsmallest(3, [top_pop_venue_ratio_df.iloc[1]['geographic area name']], keep = 'all')[top_pop_venue_ratio_df.iloc[1]['geographic area name']], '\n')
print(std_list[top_pop_venue_ratio_df.iloc[2]['cluster']].nsmallest(3, [top_pop_venue_ratio_df.iloc[2]['geographic area name']], keep = 'all')[top_pop_venue_ratio_df.iloc[2]['geographic area name']], '\n')

American Restaurant   -2.187234
Chinese Restaurant    -1.969406
Japanese Restaurant   -1.466084
Name: Moses Lake, dtype: float64 

Breakfast Spot   -2.010389
Steakhouse       -1.810329
Sandwich Place   -1.729628
Name: Aberdeen, dtype: float64 

American Restaurant   -1.438189
Thai Restaurant       -1.253129
Asian Restaurant      -1.150870
Name: Boise City, dtype: float64 



<h5>Get most underreprented restaurant type in each of the three above cities</h5>

In [47]:
top_underrepresented = [[], [], []]

In [48]:
top_underrepresented[0] = std_list[top_pop_venue_ratio_df.iloc[0]['cluster']].nsmallest(3, [top_pop_venue_ratio_df.iloc[0]['geographic area name']], keep = 'all')[top_pop_venue_ratio_df.iloc[0]['geographic area name']].index.values
top_underrepresented[1] = std_list[top_pop_venue_ratio_df.iloc[1]['cluster']].nsmallest(3, [top_pop_venue_ratio_df.iloc[1]['geographic area name']], keep = 'all')[top_pop_venue_ratio_df.iloc[1]['geographic area name']].index.values
top_underrepresented[2] = std_list[top_pop_venue_ratio_df.iloc[2]['cluster']].nsmallest(3, [top_pop_venue_ratio_df.iloc[2]['geographic area name']], keep = 'all')[top_pop_venue_ratio_df.iloc[2]['geographic area name']].index.values

<h3>Calculate the best locations for the most underrepresented restaurants, within their respective metros</h3>

<h5>Give our selected metros an "intrametro cluster" column</h5>

In [49]:
metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']]['intrametro cluster'] = 'not yet'
metro_venue_dict[top_pop_venue_ratio_df.iloc[1]['geographic area name']]['intrametro cluster'] = 'not yet'
metro_venue_dict[top_pop_venue_ratio_df.iloc[2]['geographic area name']]['intrametro cluster'] = 'not yet'

metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']].head()

,name,category,category_id,venue_id,lat,long,intrametro cluster
0,Mulligans Restuarant,American Restaurant,4bf58dd8d48988d14e941735,4c0eecb8c6cf76b03d998151,46.935745,-119.131691,not yet
1,Super China Buffet II,Sushi Restaurant,4bf58dd8d48988d1d2941735,4cc35cab1e596dcb2a0dca67,47.137726,-119.277486,not yet
2,Tree Cafe,Sushi Restaurant,4bf58dd8d48988d1d2941735,4d2670b5849f3704cc255f41,47.129524,-119.279972,not yet
3,Thai Cuisine,Thai Restaurant,4bf58dd8d48988d149941735,4c7c72faa141b1f7b15a6b79,47.129500,-119.268656,not yet
4,The Q-N-More,BBQ Joint,4bf58dd8d48988d1df931735,5a0e2b791108ba7c13603523,47.174973,-119.318770,not yet


In [50]:
metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']]['intrametro cluster'].value_counts()['not yet']

96

<h5>Cluster Restaurants in our three metros<br/>Use DBSCAN because of non-random distribution of buildings along streets</h5>

In [51]:
resto_lat_long_list = []
resto_lat_long_list.append(metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']][['lat', 'long']].to_numpy())
resto_lat_long_list.append(metro_venue_dict[top_pop_venue_ratio_df.iloc[1]['geographic area name']][['lat', 'long']].to_numpy())
resto_lat_long_list.append(metro_venue_dict[top_pop_venue_ratio_df.iloc[2]['geographic area name']][['lat', 'long']].to_numpy())

In [52]:
eps = 0.004
dbsacn_list = []
dbsacn_list.append(DBSCAN(eps = eps, min_samples = 5).fit(resto_lat_long_list[0]))
dbsacn_list.append(DBSCAN(eps = eps, min_samples = 5).fit(resto_lat_long_list[1]))
dbsacn_list.append(DBSCAN(eps = eps, min_samples = 5).fit(resto_lat_long_list[2]))

dbsacn_list[0].labels_[0:4]

array([-1,  0,  1,  1], dtype=int64)

In [53]:
metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']]['intrametro cluster'] = dbsacn_list[0].labels_
metro_venue_dict[top_pop_venue_ratio_df.iloc[1]['geographic area name']]['intrametro cluster'] = dbsacn_list[1].labels_
metro_venue_dict[top_pop_venue_ratio_df.iloc[2]['geographic area name']]['intrametro cluster'] = dbsacn_list[2].labels_

metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']].head()

,name,category,category_id,venue_id,lat,long,intrametro cluster
0,Mulligans Restuarant,American Restaurant,4bf58dd8d48988d14e941735,4c0eecb8c6cf76b03d998151,46.935745,-119.131691,-1
1,Super China Buffet II,Sushi Restaurant,4bf58dd8d48988d1d2941735,4cc35cab1e596dcb2a0dca67,47.137726,-119.277486,0
2,Tree Cafe,Sushi Restaurant,4bf58dd8d48988d1d2941735,4d2670b5849f3704cc255f41,47.129524,-119.279972,1
3,Thai Cuisine,Thai Restaurant,4bf58dd8d48988d149941735,4c7c72faa141b1f7b15a6b79,47.129500,-119.268656,1
4,The Q-N-More,BBQ Joint,4bf58dd8d48988d1df931735,5a0e2b791108ba7c13603523,47.174973,-119.318770,-1


<h5>Visualise top 10 most populated clusters</h5>

In [54]:
def getMostPopulatedClusters(metro, topXclusters = 10, viewOutliers = False):
    
    cluster_value_counts = pd.DataFrame(metro_venue_dict[metro]['intrametro cluster'].value_counts())
    clusters = list(cluster_value_counts[0:topXclusters].index)

    #Remove -1 (outliers) from clusters
    if -1 in clusters and viewOutliers == False:   
        outlier_index = clusters.index(-1)
        
        if len(cluster_value_counts) > topXclusters:
            clusters[outlier_index] = cluster_value_counts.index[topXclusters]
        else:
            clusters.pop(outlier_index)
        
    return clusters

In [55]:
clusters_list = []
clusters_list.append(getMostPopulatedClusters(top_pop_venue_ratio_df.iloc[0]['geographic area name']))
clusters_list.append(getMostPopulatedClusters(top_pop_venue_ratio_df.iloc[1]['geographic area name']))
clusters_list.append(getMostPopulatedClusters(top_pop_venue_ratio_df.iloc[2]['geographic area name']))

In [56]:
k_list = []
k_list.append(len(clusters_list[0]))
k_list.append(len(clusters_list[1]))
k_list.append(len(clusters_list[2]))

In [57]:
def visualiseVenueClusters(metro, k, clusters, mapType = "Stamen Toner"):
    
    map_ = folium.Map(width = '100%',
                      height = '100%',
                      location =  metro_df[metro_df['geographic area name'] == metro]['lat_long'].values[0][0],
                      zoom_start = 9,
                      tiles = mapType)

    x = np.arange(k)
    ys = [i + x + (i*x)**2 for i in range(k)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []

    for i in metro_venue_dict[metro].index:

        cluster = metro_venue_dict[metro]['intrametro cluster'][i]
        if cluster in clusters:

            lat = metro_venue_dict[metro]['lat'][i]
            long = metro_venue_dict[metro]['long'][i]
            venue = metro_venue_dict[metro]['name'][i]
            category = metro_venue_dict[metro]['category'][i]
            cluster_index = clusters.index(metro_venue_dict[metro]['intrametro cluster'][i])

            #label = folium.Popup(str(metro) + '\nCluster: ' + str(cluster), parse_html=True)
            label = folium.Popup(str(venue) + ': ' + str(category) +'\n\nCluster: ' + str(cluster), parse_html=True)

            folium.CircleMarker(
                [lat, long],
                radius=3,
                popup=label,
                color=rainbow[cluster_index],
                fill=True,
                fill_color=rainbow[cluster_index],
                fill_opacity=0.5).add_to(map_)

    return map_
    

In [58]:
visualiseVenueClusters(top_pop_venue_ratio_df.iloc[0]['geographic area name'], k_list[0], clusters_list[0])

In [59]:
visualiseVenueClusters(top_pop_venue_ratio_df.iloc[1]['geographic area name'], k_list[1], clusters_list[1])

In [60]:
visualiseVenueClusters(top_pop_venue_ratio_df.iloc[2]['geographic area name'], k_list[2], clusters_list[2])

<h5>Find centroids of clusters</h5>

In [61]:
def findCentroid(an_array):
   
    count = an_array.shape[0]
    
    sum_lat = np.sum(an_array[:, 0])
    sum_long = np.sum(an_array[:, 1])
    
    lat_centroid = sum_lat/count
    long_centroid = sum_long/count
    
    return lat_centroid, long_centroid

In [62]:
def findClusterCentroid(metro_venue_df, metro_clusters):

    venuesInTopClusters_df = metro_venue_df[metro_venue_df['intrametro cluster'].isin(metro_clusters)]

    centroid_list = []
    for cluster in metro_clusters:

        cluster_lat_long_array = venuesInTopClusters_df[venuesInTopClusters_df['intrametro cluster'] == cluster][['lat', 'long']].to_numpy()
        centroid_list.append([findCentroid(cluster_lat_long_array)])

    return centroid_list

In [63]:
centroid_list = []
centroid_list.append(findClusterCentroid(metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']], clusters_list[0]))
centroid_list.append(findClusterCentroid(metro_venue_dict[top_pop_venue_ratio_df.iloc[1]['geographic area name']], clusters_list[1]))
centroid_list.append(findClusterCentroid(metro_venue_dict[top_pop_venue_ratio_df.iloc[2]['geographic area name']], clusters_list[2]))

centroid_list[0][0:5]

[[(47.13048747141753, -119.27516724938005)],
 [(47.140745718456074, -119.27763669091223)],
 [(47.10257669189927, -119.24815060358323)],
 [(47.12490656544952, -119.29005985678891)]]

In [64]:
centroidDF_list = []

for i in range(k):    
    centroid_df = pd.DataFrame(clusters_list[i], pd.Series(centroid_list[i])).reset_index()
    centroid_df.rename(columns = {'index' : 'lat_long', 0 : 'cluster'}, inplace = True)
    centroidDF_list.append(centroid_df)

# seattle_centroid_df = pd.DataFrame(clusters_seattle, pd.Series(seattle_centroid_list)).reset_index()
# seattle_centroid_df.rename(columns = {'index' : 'lat_long', 0 : 'cluster'}, inplace = True)

# moses_lake_centroid_df = pd.DataFrame(clusters_moses_lake, pd.Series(moses_lake_centroid_list)).reset_index()
# moses_lake_centroid_df.rename(columns = {'index' : 'lat_long', 0 : 'cluster'}, inplace = True)

centroidDF_list[0]

,lat_long,cluster
0,"[(47.13048747141753, -119.27516724938005)]",1
1,"[(47.140745718456074, -119.27763669091223)]",0
2,"[(47.10257669189927, -119.24815060358323)]",2
3,"[(47.12490656544952, -119.29005985678891)]",3


<h5>Find clusters that don't contain the underrepresented restaurant type<br/>Find clusters that are furtherest away from that restaurant type<h5>

In [77]:
# venues_incluster_list = []
# clusters_with_top_list = []
# clusters_wout_top_list = []
# min_distance_list = []

# top_venues_list = []
# min_distance_df_list =[]

# for i in range(k):

#     venues_incluster_list.append(metro_venue_dict[top_pop_venue_ratio_df.iloc[i]['geographic area name']][(metro_venue_dict[top_pop_venue_ratio_df.iloc[i]['geographic area name']]['intrametro cluster'].isin(clusters_list[i]))])
#     clusters_with_top_list.append(venues_incluster_list[i][venues_incluster_list[i]['category'] == top_underrepresented[i][0]]['intrametro cluster'])
#     clusters_wout_top_list.append(pd.Series(clusters_list[i])[pd.Series(clusters_list[i]).isin(clusters_with_top_list[i]) == False].values)

#     min_distance_list.append([])
#     top_venues_list.append(venues_incluster_list[i][venues_incluster_list[i]['category'] == top_underrepresented[i][0]])
#     if len(top_venues_list) > 0:
#         for cluster in clusters_wout_top_list[i]:

#             (lat_0, long_0) = centroidDF_list[i][centroidDF_list[i]['cluster'] == cluster]['lat_long'].values[0][0]
#             distance_min = (top_venues_list[i].apply(lambda x: geodesic((lat_0, long_0), (x['lat'], x['long'])).meters, axis = 1).min())
#             print(distance_min)#***TEST***TEST***
#             min_distance_list[i].append([cluster, distance_min])
#             min_distance_df_list.append(pd.DataFrame(min_distance_list[i]).set_index(0).rename(columns = {1 : 'distance'}))
    

venues_incluster_0 = metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']][(metro_venue_dict[top_pop_venue_ratio_df.iloc[0]['geographic area name']]['intrametro cluster'].isin(clusters_list[0]))]
clusters_with_top_0 = venues_incluster_0[venues_incluster_0['category'] == top_underrepresented[0][0]]['intrametro cluster']
clusters_wout_top_0 = pd.Series(clusters_list[0])[pd.Series(clusters_list[0]).isin(clusters_with_top_0) == False].values

min_distance_list_0 = []
if len(clusters_wout_top_0) > 0:
    recommendedClusters = clusters_wout_top_0
else:
    recommendedClusters = clusters_with_top_0
    
for cluster in recommendedClusters:
    
    (lat_0, long_0) = centroidDF_list[0][centroidDF_list[0]['cluster'] == cluster]['lat_long'].values[0][0]    
    top_venues_0 = venues_incluster_0[venues_incluster_0['category'] == top_underrepresented[0][0]]
    distance_min = top_venues_0.apply(lambda x: geodesic((lat_0, long_0), (x['lat'], x['long'])).meters, axis = 1).min()
    if str(type(distance_min)) == "<class 'pandas.core.series.Series'>":
        distance_min = -1
    min_distance_list_0.append([cluster, distance_min])
    min_distance_df_0 = pd.DataFrame(min_distance_list_0).set_index(0).rename(columns = {1 : 'distance'})
    
venues_incluster_1 = metro_venue_dict[top_pop_venue_ratio_df.iloc[1]['geographic area name']][(metro_venue_dict[top_pop_venue_ratio_df.iloc[1]['geographic area name']]['intrametro cluster'].isin(clusters_list[1]))]
clusters_with_top_1 = venues_incluster_1[venues_incluster_1['category'] == top_underrepresented[1][0]]['intrametro cluster']
clusters_wout_top_1 = pd.Series(clusters_list[1])[pd.Series(clusters_list[1]).isin(clusters_with_top_1) == False].values

min_distance_list_1 = []
if len(clusters_wout_top_1) > 0:
    recommendedClusters = clusters_wout_top_1
else:
    recommendedClusters = clusters_with_top_1
    
for cluster in recommendedClusters:
    
    (lat_0, long_0) = centroidDF_list[1][centroidDF_list[1]['cluster'] == cluster]['lat_long'].values[0][0]    
    top_venues_1 = venues_incluster_1[venues_incluster_1['category'] == top_underrepresented[1][0]]
    distance_min = top_venues_1.apply(lambda x: geodesic((lat_0, long_0), (x['lat'], x['long'])).meters, axis = 1).min()    
    if str(type(distance_min)) == "<class 'pandas.core.series.Series'>":
        distance_min = -1    
    min_distance_list_1.append([cluster, distance_min])
    min_distance_df_1 = pd.DataFrame(min_distance_list_1).set_index(0).rename(columns = {1 : 'distance'})
    
venues_incluster_2 = metro_venue_dict[top_pop_venue_ratio_df.iloc[2]['geographic area name']][(metro_venue_dict[top_pop_venue_ratio_df.iloc[2]['geographic area name']]['intrametro cluster'].isin(clusters_list[2]))]
clusters_with_top_2 = venues_incluster_2[venues_incluster_2['category'] == top_underrepresented[2][1]]['intrametro cluster']
clusters_wout_top_2 = pd.Series(clusters_list[2])[pd.Series(clusters_list[2]).isin(clusters_with_top_2) == False].values

min_distance_list_2 = []
if len(clusters_wout_top_2) > 0:
    recommendedClusters = clusters_wout_top_2
else:
    recommendedClusters = clusters_with_top_2
    
for cluster in recommendedClusters:
    
    (lat_0, long_0) = centroidDF_list[2][centroidDF_list[2]['cluster'] == cluster]['lat_long'].values[0][0]    
    top_venues_2 = venues_incluster_2[venues_incluster_2['category'] == top_underrepresented[2][1]]
    distance_min = top_venues_2.apply(lambda x: geodesic((lat_0, long_0), (x['lat'], x['long'])).meters, axis = 1).min()
    if str(type(distance_min)) == "<class 'pandas.core.series.Series'>":
        distance_min = -1
    min_distance_list_2.append([cluster, distance_min])
    min_distance_df_2 = pd.DataFrame(min_distance_list_2).set_index(0).rename(columns = {1 : 'distance'})

In [78]:
def visualiseRecommendation(metro, topVenues_df, centroid_df, minDistance_df, category, mapType = "Stamen Toner"):
    
    map_ = folium.Map(width = '100%',
                      height = '100%',
                      location =  metro_df[metro_df['geographic area name'] == metro]['lat_long'].values[0][0],
                      zoom_start = 9,
                      tiles = mapType)

    for i in topVenues_df.index:

        lat = metro_venue_dict[metro]['lat'][i]
        long = metro_venue_dict[metro]['long'][i]
        venue = metro_venue_dict[metro]['name'][i]

        #label = folium.Popup(str(metro) + '\nCluster: ' + str(cluster), parse_html=True)
        label = folium.Popup(str(venue) + ': ' + str(category), parse_html=True)

        folium.CircleMarker(
            [lat, long],
            radius = 5,
            popup = label,
            color = '#FF0000',
            fill = True,
            fill_color = '#FF0000',
            fill_opacity = 1).add_to(map_)
        
    for i in minDistance_df.index:
        
        lat = centroid_df[centroid_df['cluster'] == i]['lat_long'].values[0][0][0]
        long = centroid_df[centroid_df['cluster'] == i]['lat_long'].values[0][0][1]
        distance = minDistance_df.loc[i]['distance']
        
        if distance == -1:
            label = folium.Popup('Open ' + str(category) + ' here: ' + str(round(lat, 4)) + ', ' + str(round(long, 4)) + '<br>No ' + str(category) + ' in metro.', max_width = 400)
        else:
            label = folium.Popup('Open ' + str(category) + ' here: ' + str(round(lat, 4)) + ', ' + str(round(long, 4)) + '<br>Nearest ' + str(category) + ' is ' + str(round(distance / 1000, 3)) + 'km away.', max_width = 400)
        
        folium.Circle(
            [lat, long],
            radius = 500,
            popup = label,
            color = '#6EB5FF',
            fill = True,
            fill_color = '#6EB5FF',
            fill_opacity = 0.3).add_to(map_)
        
    return map_

In [79]:
visualiseRecommendation(top_pop_venue_ratio_df.iloc[0]['geographic area name'], top_venues_0, centroidDF_list[0], min_distance_df_0, top_underrepresented[0][0])

In [80]:
visualiseRecommendation(top_pop_venue_ratio_df.iloc[1]['geographic area name'], top_venues_1, centroidDF_list[1], min_distance_df_1, top_underrepresented[1][0])

In [82]:
visualiseRecommendation(top_pop_venue_ratio_df.iloc[2]['geographic area name'], top_venues_2, centroidDF_list[2], min_distance_df_2, top_underrepresented[2][1])